In [6]:
import random
import time
import functools

### Decorators

In [22]:
def foo():
    time.sleep(random.randint(1, 2))

print('BEFORE')
foo()
print('AFTER')

BEFORE
AFTER


In [23]:
def foo2():
    start = time.time()

    time.sleep(random.randrange(0, 1))

    end = time.time()
    print(f'Elapsed {end - start}ms')

foo2()

Elapsed 2.002716064453125e-05ms


In [24]:
def foo3():
    start = time.time()

    lst = [i for i in range(10**6)]

    print(f'Elapsed {time.time() - start}ms')

foo3()

Elapsed 0.035893917083740234ms


In [25]:
def profile(f):
    start = time.time()

    f()

    print(f'Elapsed {time.time() - start}ms')

profile(foo)

Elapsed 2.001828193664551ms


In [31]:
type(foo())

NoneType

In [1]:
def say_hello():
    def internal():
        print('Hello')
    return internal

f = say_hello()
print(type(f))
f()

<class 'function'>
Hello


In [34]:
def say_hello():
    def internal(msg):
        print('Hello', msg)
    return internal

f1 = say_hello()
f1('John')

f1 = say_hello()
f1('Bill')

Hello John
Hello Bill


In [8]:
def say_hello(greeting='Hello'):
    def internal(msg):
        print(greeting, msg)
    return internal

f1 = say_hello()
f1('John')

f1 = say_hello('Hi')
f1('Bill')

Hello John
Hi Bill


In [9]:
def say_hello(greeting='Hello'):
    def internal(*args):
        print(greeting, *args)
    return internal

f1 = say_hello()
f1('John', 42, [1, 2, 3], True)

Hello John 42 [1, 2, 3] True


In [46]:
def profile(f):
    def deco(*args):
        start = time.time()
        result = f(*args)
        print(f'Elapsed time for function {f.__name__}: {time.time() - start}ms')
        return result
    return deco

In [37]:
# foo()
foo=profile(foo)
foo()

Elapsed time for function foo: 1.001065731048584ms
Elapsed time for function deco: 1.0011940002441406ms


In [48]:
@profile  # -> foo5 = profile(foo5)
def foo5():
    time.sleep(random.randint(1, 2))
    return 42

# foo5 = profile(foo5)
print(foo5())

42


### We need to go deeper

In [13]:
@profile
def foo6():
    time.sleep(random.randint(1, 2))
    return 42

print(foo6())

Elapsed time for function foo6: 2.0023670196533203ms
None


In [14]:
def profile(f):
    def deco(*args):
        start = time.time()
        result = f(*args)
        print(f'Elapsed time for function {f.__name__}: {time.time() - start}ms')
        return result
    return deco

In [51]:
@profile
def foo7():
    """Help for foo7"""
    time.sleep(random.randint(1, 2))
    return 42

print(foo7())
help(foo7)

Elapsed time for function foo7: 2.001713275909424ms
42
Help on function deco in module __main__:

deco(*args)



In [52]:
def profile(f):
    @functools.wraps(f)
    def deco(*args):
        start = time.time()
        result = f(*args)
        print(f'Elapsed time for function {f.__name__}: {time.time() - start}ms')
        return result
    return deco

In [54]:
@profile
def foo8():
    """Help for foo8"""
    time.sleep(random.randint(1, 2))
    return 42

print(foo8())
help(foo8)

Elapsed time for function foo8: 1.0017116069793701ms
42
Help on function foo8 in module __main__:

foo8()
    Help for foo8



In [56]:
@profile('Time spent') # -> profile_ = profile('Time spent')
                       # -> foo5 = profile_(foo5)

def foo8():
    """Help for foo7"""
    time.sleep(random.randint(1, 2))
    return 42

print(foo8())
help(foo8)

Time spent foo8: 2.0019683837890625ms
42
Help on function foo8 in module __main__:

foo8()
    Help for foo7



In [55]:
def profile(msg):
    def profile_(f):
        @functools.wraps(f)
        def deco(*args):
            start = time.time()
            result = f(*args)
            print(msg, f'{f.__name__}: {time.time() - start}ms')
            return result
        return deco
    return profile_

In [84]:
def profile(msg="Elapsed time for function"):
    def internal(f):
        @functools.wraps(f)
        def deco(*args):
            start = time.time()
            deco._num_call += 1
            result = f(*args)
            deco._num_call -= 1
            
            if deco._num_call == 0:
                print(msg, f'{f.__name__}: {time.time() - start}ms')
            return result
        
        deco._num_call = 0
        return deco
    
    return internal

In [57]:
def repeate(n=2):
    def internal(f):
        @functools.wraps(f)
        def repeater(*args, **kwargs):
            for _ in range(n):
                f(*args, **kwargs)
        return repeater
    return internal

@repeate(3)
def my_print(*args):
    print(*args)

my_print('Hello')


Hello
Hello
Hello


In [61]:
@profile()
@repeate()
def foo9():
    """Help for foo9"""
    time.sleep(random.randint(1, 2))
    return 42

foo9()

Elapsed time for function foo9: 3.0023088455200195ms


In [15]:
def cache(f):
    @functools.wraps(f)
    def deco(*args):
#         if args not in deco._cache:
#             result = f(*args)
#             deco._cache[args] = result
#         return deco._cache[args]
        if args in deco._cache:
            return deco._cache[args]
        result = f(*args)
        deco._cache[args] = result
        return result


    deco._cache = {}
    return deco

In [85]:
@profile()
@cache(max_limit=64)
# 0 1 1 2 3 5 8 13 ...
def fibo(n):
    """Inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)

    
@profile()
@cache
def foo(n):
    time.sleep(n)

# foo(5)
# foo(6)

In [86]:
print(5, '->', fibo(1000))

Elapsed time for function fibo: 1.1205673217773438e-05ms
5 -> 5


### Magic attributes

In [16]:
class MyInt(int):
        
    def __add__(self, value):
        return super().__add__(int(value))
    
val = MyInt(42)
val + '1'


43

In [13]:
class context_manager():
    def __enter__(self):
        print('ENTER')
        return None

    def __exit__(self, type, value, traceback):
        print('EXIT')
        
with context_manager() as cm:
    print('hello')

ENTER
hello
EXIT


In [2]:
class coloured_print():
    
    def __init__(self):#, colour="31;40m"):
        self.old_print = None

    def __enter__(self):
        def my_print(*args):
            self.old_print('\x1B[31;40m', *args, '\x1B[0m')
        global print
        self.old_print = print
        print = my_print

    def __exit__(self, type, value, traceback):
        global print
        print = self.old_print
        
print('BEFORE')
with coloured_print() as aa:
    print('Hello')
    print('world', 42, [1, 2, 3])
print('AFTER')

BEFORE
 Hello 
 world 42 [1, 2, 3] 
AFTER


In [4]:
class timer():
    def __init__(self, message):
        self.message = message

    def __enter__(self):
        self.start = time.time()
        return None

    def __exit__(self, type, value, traceback):
        elapsed_time = (time.time() - self.start)
        print(self.message.format(elapsed_time))

In [7]:
with timer('Elapsed: {}s'):
    time.sleep(1)

Elapsed: 1.001065969467163s


In [11]:
with open('test.txt', 'w+') as f:
    f.write('abc')
    
with open('test.txt', 'r') as f:
    print(f.read())

abc


In [ ]:
f = open('test.txt', 'r')
# 10/0
f.close()